# ロボットマニピュレータの逆運動学を4×4マトリクスを用いて解く！！！

Created by Hiroki Miyamoto 

Modified by Masahiro Furukawa, Aug 18, 2020

In [110]:
#参考URL -> https://qiita.com/tibigame/items/61cecf86fc978628bfee
#参考図書 -> ポールのロボット・マニピュレータ

import numpy as np
import sympy as sym
sym.init_printing()
Pi = sym.S.Pi # 円周率
E = sym.S.Exp1 # 自然対数の底
I = sym.S.ImaginaryUnit # 虚数単位

#sympyの円周率の方を使うことをすすめる（こっちの方が量子化誤差が大きくなる．numpyも同様に大きい）
import math
pi = math.pi

# 使用する変数の定義(小文字1文字は全てシンボルとする)
(a,b,c,d,e,f,g,h,i,j,k,l,m,n,o,p,q,r,s,t,u,v,w,x,y,z) = sym.symbols('a b c d e f g h i j k l m n o p q r s t u v w x y z')
#a～fは角度変数

(n_x, n_y, n_z, o_x, o_y, o_z, a_x, a_y, a_z, p_x, p_y, p_z) = sym.symbols('n_x, n_y, n_z, o_x, o_y, o_z, a_x, a_y, a_z, p_x, p_y, p_z')


In [111]:
#sin.cosの簡易記述用
def S(a):
    return sym.sin(a)

def C(a):
    return sym.cos(a)

In [112]:
#回転・並進行列
def rotx(a):
    return sym.Matrix([[1, 0, 0, 0], [0, C(a), -S(a), 0], [0, S(a), C(a), 0], [0, 0, 0, 1]])

def roty(a):
    return sym.Matrix([[C(a), 0, S(a), 0], [0, 1, 0, 0], [-S(a), 0, C(a), 0], [0, 0, 0, 1]])

def rotz(a):
    return sym.Matrix([[C(a), -S(a), 0, 0], [S(a), C(a), 0, 0], [0, 0, 1, 0], [0, 0, 0, 1]])

def trans(x, y, z):
    return sym.Matrix([[1, 0, 0, x], [0, 1, 0, y], [0, 0, 1, z], [0, 0, 0, 1]])


### DH法に基づく座標系間の関係表現

|座標系 i|Z_i-1軸回りに角度θ_i|Z_i-1に沿って距離d_iだけ並進|回転後のX_i-1 (=X_i)に沿って長さa_iだけ並進|X_i軸周りにねじれ角α_iだけ回転|
|-|-|-|-|-|
|1|a|h|g|Pi/2|
|2|b|0|0|0|(yにj)
|3|c|0|-k|-Pi/2|(cの初期位置は-Pi/2)
|4|d|l|0|Pi/2|
|5|e|0|0|-Pi/2|(yにm)
|6|f|n|0|0|

座標系0→1：
Z0軸方向にd1だけ平行移動してから、関節１の関節角度θ1だけ回転、a1=0、α1=0

座標系1→2：
X1軸方向にa2移動、関節２の角度θ2だけ回転、α2=0、d2=0

座標系2→3：
X2軸方向にa3移動、X2軸回りにα3=180度(π)回転、回転後のZ2軸方向(鉛直下向)にd3移動、θ3=0

座標系3→4：
Z0軸方向にd1だけ平行移動してから、関節１の関節角度θ1だけ回転、a1=0、α1=0

座標系4→5：
X1軸方向にa2移動、関節２の角度θ2だけ回転、α2=0、d2=0

座標系5→6：
X2軸方向にa3移動、X2軸回りにα3=180度(π)回転、回転後のZ2軸方向(鉛直下向)にd3移動、θ3=0

In [113]:
#変換行列の指定（1.モーターによる回転成分 -> 2.機構に基づく並進成分 -> 3.機構に基づく座標系の回転）

A1=rotz(a)*trans(g,0,h)*rotx(Pi/2)
sym.trigsimp(A1)

⎡cos(a)  0  sin(a)   g⋅cos(a)⎤
⎢                            ⎥
⎢sin(a)  0  -cos(a)  g⋅sin(a)⎥
⎢                            ⎥
⎢  0     1     0        h    ⎥
⎢                            ⎥
⎣  0     0     0        1    ⎦

In [123]:
# 逆行列
A1inv=rotx(-Pi/2)*trans(-g,0,-h)*rotz(-a)
sym.trigsimp(A1inv)

SyntaxError: invalid syntax (<ipython-input-123-fba34da2b59c>, line 3)

In [121]:
# 逆行列をかけると単位行列になることの確認
ret = A1inv*A1
sym.trigsimp(ret)

⎡1  0  0  0⎤
⎢          ⎥
⎢0  1  0  0⎥
⎢          ⎥
⎢0  0  1  0⎥
⎢          ⎥
⎣0  0  0  1⎦

In [5]:
#変換行列の指定（1.モーターによる回転成分 -> 2.機構に基づく並進成分 -> 3.機構に基づく座標系の回転）

A2=rotz(b)*trans(0,j,0)
sym.trigsimp(A2)

⎡cos(b)  -sin(b)  0  -j⋅sin(b)⎤
⎢                             ⎥
⎢sin(b)  cos(b)   0  j⋅cos(b) ⎥
⎢                             ⎥
⎢  0        0     1      0    ⎥
⎢                             ⎥
⎣  0        0     0      1    ⎦

In [6]:
#変換行列の指定（1.モーターによる回転成分 -> 2.機構に基づく並進成分 -> 3.機構に基づく座標系の回転）

A3=rotz(c)*trans(0,k,0)*roty(Pi/2)
sym.trigsimp(A3)

⎡0   -sin(c)  cos(c)  -k⋅sin(c)⎤
⎢                              ⎥
⎢0   cos(c)   sin(c)  k⋅cos(c) ⎥
⎢                              ⎥
⎢-1     0       0         0    ⎥
⎢                              ⎥
⎣0      0       0         1    ⎦

In [7]:
#変換行列の指定（1.モーターによる回転成分 -> 2.機構に基づく並進成分 -> 3.機構に基づく座標系の回転）

A4=rotz(d)*trans(0,0,l)*roty(-Pi/2)
sym.trigsimp(A4)

⎡0  -sin(d)  -cos(d)  0⎤
⎢                      ⎥
⎢0  cos(d)   -sin(d)  0⎥
⎢                      ⎥
⎢1     0        0     l⎥
⎢                      ⎥
⎣0     0        0     1⎦

In [8]:
#変換行列の指定（1.モーターによる回転成分 -> 2.機構に基づく並進成分 -> 3.機構に基づく座標系の回転）

A5=rotz(e)*trans(m,0,0)*roty(Pi/2)
sym.trigsimp(A5)

⎡0   -sin(e)  cos(e)  m⋅cos(e)⎤
⎢                             ⎥
⎢0   cos(e)   sin(e)  m⋅sin(e)⎥
⎢                             ⎥
⎢-1     0       0        0    ⎥
⎢                             ⎥
⎣0      0       0        1    ⎦

In [9]:
#変換行列の指定（1.モーターによる回転成分 -> 2.機構に基づく並進成分 -> 3.機構に基づく座標系の回転）

A6=rotz(f)*trans(0,0,n)
sym.trigsimp(A6)

⎡cos(f)  -sin(f)  0  0⎤
⎢                     ⎥
⎢sin(f)  cos(f)   0  0⎥
⎢                     ⎥
⎢  0        0     1  n⎥
⎢                     ⎥
⎣  0        0     0  1⎦

In [103]:
sym.trigsimp(A1*A2*A3*A4*A5*A6)

⎡(((-sin(b)⋅cos(a)⋅cos(c) - sin(c)⋅cos(a)⋅cos(b))⋅cos(d) + sin(a)⋅sin(d))⋅cos(
⎢                                                                             
⎢(((-sin(a)⋅sin(b)⋅cos(c) - sin(a)⋅sin(c)⋅cos(b))⋅cos(d) - sin(d)⋅cos(a))⋅cos(
⎢                                                                             
⎢                                       (-sin(b)⋅sin(c) + cos(b)⋅cos(c))⋅sin(d
⎢                                                                             
⎣                                                                             

e) - (-sin(b)⋅sin(c)⋅cos(a) + cos(a)⋅cos(b)⋅cos(c))⋅sin(e))⋅sin(f) + ((-sin(b)
                                                                              
e) - (-sin(a)⋅sin(b)⋅sin(c) + sin(a)⋅cos(b)⋅cos(c))⋅sin(e))⋅sin(f) + ((-sin(a)
                                                                              
)⋅cos(f) + ((-sin(b)⋅sin(c) + cos(b)⋅cos(c))⋅cos(d)⋅cos(e) - (sin(b)⋅cos(c) + 
                                                   

In [98]:
T5=A6
T4=A5*A6
T3=A4*A5*A6
T2=A3*A4*A5*A6
T1=A2*A3*A4*A5*A6
T=A1*A2*A3*A4*A5*A6

#x,y,zの順運動学が確認したければここを表示して，Tの４列目を確認
#sym.trigsimp(T*trans(n,0,0))

In [104]:
sym.trigsimp(T)

⎡ ((sin(a)⋅sin(d) - sin(b + c)⋅cos(a)⋅cos(d))⋅cos(e) - sin(e)⋅cos(a)⋅cos(b + c
⎢                                                                             
⎢(-(sin(a)⋅sin(b + c)⋅cos(d) + sin(d)⋅cos(a))⋅cos(e) - sin(a)⋅sin(e)⋅cos(b + c
⎢                                                                             
⎢                             (-sin(e)⋅sin(b + c) + cos(d)⋅cos(e)⋅cos(b + c))⋅
⎢                                                                             
⎣                                                                     0       

))⋅sin(f) - (sin(a)⋅cos(d) + sin(d)⋅sin(b + c)⋅cos(a))⋅cos(f)    ((sin(a)⋅sin(
                                                                              
))⋅sin(f) + (-sin(a)⋅sin(d)⋅sin(b + c) + cos(a)⋅cos(d))⋅cos(f)  (-(sin(a)⋅sin(
                                                                              
sin(f) + sin(d)⋅cos(f)⋅cos(b + c)                                             
                                                   

In [105]:
# Masahiro Furukawa
# Aug, 17, 2020
#
# refernce : https://qiita.com/JmpM/items/4bea4997aaf406cca3b4
T=sym.trigsimp(T)
for ii in range(4):
    for jj in range(4):
        idx = jj*4+ii
        code = sym.ccode(T[idx],assign_to=('Trans['+str(jj)+']['+str(ii)+']'), standard='C89')
        print(code)
    print()

Trans[0][0] = ((sin(a)*sin(d) - sin(b + c)*cos(a)*cos(d))*cos(e) - sin(e)*cos(a)*cos(b + c))*sin(f) - (sin(a)*cos(d) + sin(d)*sin(b + c)*cos(a))*cos(f);
Trans[1][0] = (-(sin(a)*sin(b + c)*cos(d) + sin(d)*cos(a))*cos(e) - sin(a)*sin(e)*cos(b + c))*sin(f) + (-sin(a)*sin(d)*sin(b + c) + cos(a)*cos(d))*cos(f);
Trans[2][0] = (-sin(e)*sin(b + c) + cos(d)*cos(e)*cos(b + c))*sin(f) + sin(d)*cos(f)*cos(b + c);
Trans[3][0] = 0;

Trans[0][1] = ((sin(a)*sin(d) - sin(b + c)*cos(a)*cos(d))*cos(e) - sin(e)*cos(a)*cos(b + c))*cos(f) + (sin(a)*cos(d) + sin(d)*sin(b + c)*cos(a))*sin(f);
Trans[1][1] = (-(sin(a)*sin(b + c)*cos(d) + sin(d)*cos(a))*cos(e) - sin(a)*sin(e)*cos(b + c))*cos(f) - (-sin(a)*sin(d)*sin(b + c) + cos(a)*cos(d))*sin(f);
Trans[2][1] = (-sin(e)*sin(b + c) + cos(d)*cos(e)*cos(b + c))*cos(f) - sin(d)*sin(f)*cos(b + c);
Trans[3][1] = 0;

Trans[0][2] = (sin(a)*sin(d) - sin(b + c)*cos(a)*cos(d))*sin(e) + cos(a)*cos(e)*cos(b + c);
Trans[1][2] = -(sin(a)*sin(b + c)*cos(d) + sin(d)*cos(a))*sin(

In [106]:
A1inv=A1.inv()
A2inv=A2.inv()
A3inv=A3.inv()
A4inv=A4.inv()
A5inv=A5.inv()
A6inv=A6.inv()

In [107]:
A1

⎡cos(a)  0  sin(a)   g⋅cos(a)⎤
⎢                            ⎥
⎢sin(a)  0  -cos(a)  g⋅sin(a)⎥
⎢                            ⎥
⎢  0     1     0        h    ⎥
⎢                            ⎥
⎣  0     0     0        1    ⎦

In [91]:
A1inv

⎡     2                            ⎤
⎢  sin (a)     1                   ⎥
⎢- ─────── + ──────  sin(a)   0  -g⎥
⎢   cos(a)   cos(a)                ⎥
⎢                                  ⎥
⎢        0              0     1  -h⎥
⎢                                  ⎥
⎢      sin(a)        -cos(a)  0  0 ⎥
⎢                                  ⎥
⎣        0              0     0  1 ⎦

In [80]:
A1inv*A1

⎡⎛     2            ⎞                      ⎛     2            ⎞               
⎢⎜  sin (a)     1   ⎟             2        ⎜  sin (a)     1   ⎟               
⎢⎜- ─────── + ──────⎟⋅cos(a) + sin (a)  0  ⎜- ─────── + ──────⎟⋅sin(a) - sin(a
⎢⎝   cos(a)   cos(a)⎠                      ⎝   cos(a)   cos(a)⎠               
⎢                                                                             
⎢                  0                    1                       0             
⎢                                                                             
⎢                                                          2         2        
⎢                  0                    0               sin (a) + cos (a)     
⎢                                                                             
⎣                  0                    0                       0             

            ⎛     2            ⎞                       ⎤
            ⎜  sin (a)     1   ⎟               2       ⎥
)⋅cos(a)  g⋅⎜- ─

In [38]:
T=sym.Matrix([[n_x, o_x, a_x, p_x], [n_y, o_y, a_y, p_y], [n_z, o_z, a_z, p_z], [0, 0, 0, 1]])
T

⎡nₓ   oₓ   aₓ   pₓ ⎤
⎢                  ⎥
⎢n_y  o_y  a_y  p_y⎥
⎢                  ⎥
⎢n_z  o_z  a_z  p_z⎥
⎢                  ⎥
⎣ 0    0    0    1 ⎦

In [59]:
B1T = B1*T
B1T

⎡   ⎛     2            ⎞                  ⎛     2            ⎞                
⎢   ⎜  sin (a)     1   ⎟                  ⎜  sin (a)     1   ⎟                
⎢nₓ⋅⎜- ─────── + ──────⎟ + n_y⋅sin(a)  oₓ⋅⎜- ─────── + ──────⎟ + o_y⋅sin(a)  a
⎢   ⎝   cos(a)   cos(a)⎠                  ⎝   cos(a)   cos(a)⎠                
⎢                                                                             
⎢                n_z                                   o_z                    
⎢                                                                             
⎢       nₓ⋅sin(a) - n_y⋅cos(a)                oₓ⋅sin(a) - o_y⋅cos(a)          
⎢                                                                             
⎣                 0                                     0                     

  ⎛     2            ⎞                       ⎛     2            ⎞             
  ⎜  sin (a)     1   ⎟                       ⎜  sin (a)     1   ⎟             
ₓ⋅⎜- ─────── + ──────⎟ + a_y⋅sin(a)  -g + pₓ⋅⎜- ───

# inverse kinematics

In [54]:
T1[10]

In [61]:
B1T[10]

In [60]:
T1[11]

In [58]:
B1T[11]

In [14]:
#逆行列確認
#sym.trigsimp(A1*B1*A2*B2*A3*B3*A4*B4*A5*B5*A6*B6*A7*B7)

In [21]:
U1=sym.trigsimp(B1)
U2=sym.trigsimp(B2*B1)
U3=sym.trigsimp(B3*B2*B1)
U4=sym.trigsimp(B4*B3*B2*B1)
U5=sym.trigsimp(B5*B4*B3*B2*B1)

In [16]:
#T.subs([(a,0),(b,0),(c,0),(d,0),(e,0),(f,0)])

In [22]:
sym.trigsimp([U1,T1])

⎡⎡cos(a)  sin(a)   0  -g⎤  ⎡-(sin(e)⋅cos(b + c) + sin(b + c)⋅cos(d)⋅cos(e))⋅si
⎢⎢                      ⎥  ⎢                                                  
⎢⎢  0        0     1  -h⎥  ⎢(-sin(e)⋅sin(b + c) + cos(d)⋅cos(e)⋅cos(b + c))⋅si
⎢⎢                      ⎥, ⎢                                                  
⎢⎢sin(a)  -cos(a)  0  0 ⎥  ⎢                      sin(d)⋅sin(f)⋅cos(e) - cos(d
⎢⎢                      ⎥  ⎢                                                  
⎣⎣  0        0     0  1 ⎦  ⎣                                        0         

n(f) - sin(d)⋅sin(b + c)⋅cos(f)  -(sin(e)⋅cos(b + c) + sin(b + c)⋅cos(d)⋅cos(e
                                                                              
n(f) + sin(d)⋅cos(f)⋅cos(b + c)  (-sin(e)⋅sin(b + c) + cos(d)⋅cos(e)⋅cos(b + c
                                                                              
)⋅cos(f)                                               sin(d)⋅cos(e)⋅cos(f) + 
                                                   

In [23]:
sym.trigsimp([U2,T2])

⎡⎡cos(a)⋅cos(b)   sin(a)⋅cos(b)   sin(b)   -g⋅cos(b) - h⋅sin(b)  ⎤  ⎡-(sin(c)⋅
⎢⎢                                                               ⎥  ⎢         
⎢⎢-sin(b)⋅cos(a)  -sin(a)⋅sin(b)  cos(b)  g⋅sin(b) - h⋅cos(b) - j⎥  ⎢(-sin(c)⋅
⎢⎢                                                               ⎥, ⎢         
⎢⎢    sin(a)         -cos(a)        0                0           ⎥  ⎢         
⎢⎢                                                               ⎥  ⎢         
⎣⎣      0               0           0                1           ⎦  ⎣         

cos(d)⋅cos(e) + sin(e)⋅cos(c))⋅sin(f) - sin(c)⋅sin(d)⋅cos(f)  -(sin(c)⋅cos(d)⋅
                                                                              
sin(e) + cos(c)⋅cos(d)⋅cos(e))⋅sin(f) + sin(d)⋅cos(c)⋅cos(f)  (-sin(c)⋅sin(e) 
                                                                              
       sin(d)⋅sin(f)⋅cos(e) - cos(d)⋅cos(f)                                   
                                                   

In [20]:
sym.trigsimp([U3,T3])

⎡⎡     -sin(a)              cos(a)            0                           0   
⎢⎢                                                                            
⎢⎢-sin(b + c)⋅cos(a)  -sin(a)⋅sin(b + c)  cos(b + c)  g⋅sin(b + c) - h⋅cos(b +
⎢⎢                                                                            
⎢⎢cos(a)⋅cos(b + c)   sin(a)⋅cos(b + c)   sin(b + c)   -g⋅cos(b + c) - h⋅sin(b
⎢⎢                                                                            
⎣⎣        0                   0               0                           1   

                  ⎤  ⎡-sin(d)⋅sin(f)⋅cos(e) + cos(d)⋅cos(f)  -sin(d)⋅cos(e)⋅co
                  ⎥  ⎢                                                        
 c) - j⋅cos(c) - k⎥  ⎢sin(d)⋅cos(f) + sin(f)⋅cos(d)⋅cos(e)   -sin(d)⋅sin(f) + 
                  ⎥, ⎢                                                        
 + c) - j⋅sin(c)  ⎥  ⎢           -sin(e)⋅sin(f)                         -sin(e
                  ⎥  ⎢                             

In [21]:
sym.trigsimp([U4,T4])

⎡⎡           cos(a)⋅cos(b + c)                          sin(a)⋅cos(b + c)     
⎢⎢                                                                            
⎢⎢sin(a)⋅sin(d) - sin(b + c)⋅cos(a)⋅cos(d)  -sin(a)⋅sin(b + c)⋅cos(d) - sin(d)
⎢⎢                                                                            
⎢⎢sin(a)⋅cos(d) + sin(d)⋅sin(b + c)⋅cos(a)  sin(a)⋅sin(d)⋅sin(b + c) - cos(a)⋅
⎢⎢                                                                            
⎣⎣                   0                                          0             

             sin(b + c)          -g⋅cos(b + c) - h⋅sin(b + c) - j⋅sin(c) - l  
                                                                              
⋅cos(a)  cos(d)⋅cos(b + c)   (g⋅sin(b + c) - h⋅cos(b + c) - j⋅cos(c) - k)⋅cos(
                                                                              
cos(d)   -sin(d)⋅cos(b + c)  (-g⋅sin(b + c) + h⋅cos(b + c) + j⋅cos(c) + k)⋅sin
                                                   

In [22]:
sym.trigsimp([U5,T5])

⎡⎡                  -sin(a)⋅cos(d) - sin(d)⋅sin(b + c)⋅cos(a)                 
⎢⎢                                                                            
⎢⎢-(sin(e)⋅cos(b + c) + sin(b + c)⋅cos(d)⋅cos(e))⋅cos(a) + sin(a)⋅sin(d)⋅cos(e
⎢⎢                                                                            
⎢⎢(-sin(e)⋅sin(b + c)⋅cos(d) + cos(e)⋅cos(b + c))⋅cos(a) + sin(a)⋅sin(d)⋅sin(e
⎢⎢                                                                            
⎣⎣                                      0                                     

                     -sin(a)⋅sin(d)⋅sin(b + c) + cos(a)⋅cos(d)                
                                                                              
)  -(sin(e)⋅cos(b + c) + sin(b + c)⋅cos(d)⋅cos(e))⋅sin(a) - sin(d)⋅cos(a)⋅cos(
                                                                              
)  (-sin(e)⋅sin(b + c)⋅cos(d) + cos(e)⋅cos(b + c))⋅sin(a) - sin(d)⋅sin(e)⋅cos(
                                                   

In [23]:
#x=T[0,3]
#y=T[1,3]
#z=T[2,3]